---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score
        
    # load data
    train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    test = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')

    # pre-processing
    
    # drop all rows with Null compliance
    train = train[np.isfinite(train['compliance'])]
    # drop all rows not in the U.S
    train = train[train.country == 'USA']
    test = test[test.country == 'USA']
    # merge latlons and addresses with data
    train = pd.merge(train, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    # drop all unnecessary columns
    train.drop(['agency_name', 'inspector_name', 'violator_name', 'non_us_str_code', 'violation_description', 
                'grafitti_status', 'state_fee', 'admin_fee', 'ticket_issued_date', 'hearing_date',
                # columns not available in test
                'payment_amount', 'balance_due', 'payment_date', 'payment_status', 
                'collection_status', 'compliance_detail', 
                # address related columns
                'violation_zip_code', 'country', 'address', 'violation_street_number',
                'violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name', 
                'city', 'state', 'zip_code', 'address'], axis=1, inplace=True)
    # discretizing relevant columns
    label_encoder = LabelEncoder()
    label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
    train['disposition'] = label_encoder.transform(train['disposition'])
    test['disposition'] = label_encoder.transform(test['disposition'])
    label_encoder = LabelEncoder()
    label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
    train['violation_code'] = label_encoder.transform(train['violation_code'])
    test['violation_code'] = label_encoder.transform(test['violation_code'])
    train['lat'] = train['lat'].fillna(train['lat'].mean())
    train['lon'] = train['lon'].fillna(train['lon'].mean())
    test['lat'] = test['lat'].fillna(test['lat'].mean())
    test['lon'] = test['lon'].fillna(test['lon'].mean())
    train_columns = list(train.columns.values)
    train_columns.remove('compliance')
    test = test[train_columns]
    
    # train the model
    
    X_train, X_test, y_train, y_test = train_test_split(train.ix[:, train.columns != 'compliance'], train['compliance'])
    regr_rf = RandomForestRegressor()
    grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
    grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    
    return pd.DataFrame(grid_clf_auc.predict(test), test.ticket_id)

In [2]:
#blight_model()

In [3]:
train = pd.read_csv('train.csv', encoding = "ISO-8859-1", parse_dates = ['ticket_issued_date', 'hearing_date'])
test = pd.read_csv('readonly/test.csv', parse_dates = ['ticket_issued_date', 'hearing_date'])

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Drop Column with all NA values
train.dropna(how='all', axis = 1,inplace = True)

In [5]:
# Check memory usage 
train.memory_usage().sum()

66080864

It can be made more efficient by reducing un-necessary memory usage.This can be done grpouping similar things into categories

In [6]:
# Reduce memory usage by casting

for i in range(len(train.columns)):
    if len(train[train.columns[i]].unique()) < 250:
        train[train.columns[i]] = train[train.columns[i]] .astype('category')

More than half memory is used saved!!

In [7]:
train.memory_usage().sum()

33296058

Now lets drop all things with Null values in it

In [8]:
#drop all with nan
train = train.dropna(axis = 1,how  = 'all')
train = train.dropna(axis = 0,how  = 'all')

We can see that many fields have many missing datas so let's get rid of columns with more than 30 % missing data

In [9]:
# Handling Missing data
total_null = train.isnull().sum().sort_values(ascending = False)
ratio = (train.isnull().sum()/train.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total_null, ratio], axis=1, keys = ['Total', 'Percent'])

missing_data

,Total,Percent
grafitti_status,250305,0.999996
non_us_str_code,250303,0.999988
collection_status,213409,0.852592
payment_date,209193,0.835749
compliance,90426,0.361262
hearing_date,12491,0.049903
mailing_address_str_number,3602,0.014390
state,93,0.000372
violator_name,34,0.000136
mailing_address_str_name,4,0.000016


In [10]:
train.drop(missing_data[missing_data['Percent'] > 0.5].index, axis=1, inplace=True)

In [11]:
#drop same columns from test data too
missing_data[missing_data['Percent']>0.5].index

Index(['grafitti_status', 'non_us_str_code', 'collection_status',
       'payment_date'],
      dtype='object')

Above columns are worthless as more than 50% of time they are NaN so they are not good for predicting

Now lets delete the columns which have all values same

In [12]:
#remove all columns of equal values...

length = len(train.columns)
for i in range (length):
    if len(train[train.columns[i]].unique()) == 1:
         print(train.columns[i])


clean_up_cost


Only 'clean_up_cost' has all the values same (i.e = 0) so it is also independent of our target variable as

In [13]:
train.drop(labels = 'clean_up_cost' ,axis = 1,inplace = True)
test.drop(labels = 'clean_up_cost' ,axis = 1,inplace = True)

Let's see how many more null columns are there

In [14]:
train.isnull().sum()

ticket_id                         0
agency_name                       0
inspector_name                    0
violator_name                    34
violation_street_number           0
violation_street_name             0
mailing_address_str_number     3602
mailing_address_str_name          4
city                              0
state                            93
zip_code                          1
country                           0
ticket_issued_date                0
hearing_date                  12491
violation_code                    0
violation_description             0
disposition                       0
fine_amount                       1
admin_fee                         0
state_fee                         0
late_fee                          0
discount_amount                   0
judgment_amount                   0
payment_amount                    0
balance_due                       0
payment_status                    0
compliance_detail                 0
compliance                  

street number , street name can be null as people have different format each time but are they relevant??

In [15]:
address = pd.read_csv('readonly/addresses.csv')
latlons = pd.read_csv('readonly/latlons.csv')

In [16]:
address = address.set_index('address').join(latlons.set_index('address'), how='left')
address.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [17]:
train = train.set_index('ticket_id').join(address.set_index('ticket_id'))
test = test.set_index('ticket_id').join(address.set_index('ticket_id'))

In [18]:
train.columns

Index(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'judgment_amount', 'payment_amount', 'balance_due', 'payment_status',
       'compliance_detail', 'compliance', 'lat', 'lon'],
      dtype='object')

Now List of Items I think are irrevant..... As all this info should be covered in lan and log are below

In [19]:
list_to_remove = [ 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country']

In [21]:
train.drop(list_to_remove,axis = 1,inplace = True)
test.drop(list_to_remove,axis = 1,inplace = True)

In [29]:
test.drop(['grafitti_status', 'non_us_str_code'], axis=1, inplace=True)

Let's reduce more feature,we can just use Total amount to pay instead of below features

In [31]:
train.columns

Index(['agency_name', 'inspector_name', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'judgment_amount', 'payment_amount', 'balance_due', 'payment_status',
       'compliance_detail', 'compliance', 'lat', 'lon'],
      dtype='object')

In [32]:
train['Total_amount_to_pay'] = train['fine_amount'] + train['admin_fee'] + train['state_fee'] + train['late_fee'] - train['discount_amount']
test['Total_amount_to_pay'] = test['fine_amount'] + test['admin_fee'] + test['state_fee'] + test['late_fee'] - test['discount_amount']

In [33]:
list_drop = ['fine_amount','admin_fee','state_fee','late_fee','discount_amount']
train.drop(list_drop,axis = 1,inplace = True)
test.drop(list_drop,axis = 1,inplace = True)

In [34]:
train.columns

Index(['agency_name', 'inspector_name', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition',
       'judgment_amount', 'payment_amount', 'balance_due', 'payment_status',
       'compliance_detail', 'compliance', 'lat', 'lon', 'Total_amount_to_pay'],
      dtype='object')

In [35]:
test.columns

Index(['agency_name', 'inspector_name', 'violation_zip_code',
       'ticket_issued_date', 'hearing_date', 'violation_code',
       'violation_description', 'disposition', 'judgment_amount', 'lat', 'lon',
       'Total_amount_to_pay'],
      dtype='object')

Now lets fill NaN values of above calculated fields to train the model,As they are very less compared to original dataset we can drop them too,but dropping them from testSet isn't allowed so let's fill them

In [36]:
train.dropna(subset = ['lat','lon','Total_amount_to_pay'],inplace = True)
test['lat'].fillna(test.lat.mean(),inplace = True)
test['lon'].fillna(test.lon.mean(),inplace = True)
test.isnull().sum()

agency_name                  0
inspector_name               0
violation_zip_code       36977
ticket_issued_date           0
hearing_date              2197
violation_code               0
violation_description        0
disposition                  0
judgment_amount              0
lat                          0
lon                          0
Total_amount_to_pay          0
dtype: int64

Now time_gap between dates is only needed so let's calculate that

In [41]:
train['time_gap'] = train['hearing_date'] - train['ticket_issued_date']
train['time_gap'] = train['time_gap'].dt.days
train['time_gap'].fillna(73, inplace = True)
train.drop(['hearing_date','ticket_issued_date'], axis = 1, inplace = True)

test['time_gap'] = test['hearing_date'] - test['ticket_issued_date']
test['time_gap'] = test['time_gap'].dt.days
test['time_gap'].fillna(73, inplace = True)
test.drop(['hearing_date','ticket_issued_date'], axis = 1, inplace = True)

Now dropping features that aren't important and creating features from string categories

In [45]:
train.drop(['violation_description','payment_amount','balance_due','payment_status','compliance_detail','inspector_name'],axis = 1,inplace = True)

In [47]:
train.columns

Index(['agency_name', 'violation_code', 'disposition', 'judgment_amount',
       'compliance', 'lat', 'lon', 'Total_amount_to_pay', 'time_gap'],
      dtype='object')

In [50]:
test.drop(['inspector_name', 'violation_zip_code', 'violation_description'], axis=1, inplace=True)

In [51]:
string_features = ['disposition','agency_name']
train =  pd.get_dummies(train, columns = string_features, drop_first = True)
test =  pd.get_dummies(test,columns = string_features, drop_first = True)

In [53]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
train['violation_code'] = label_encoder.transform(train['violation_code'])
test['violation_code'] = label_encoder.transform(test['violation_code'])

Taking only NON NaN values for prediction

In [55]:
train = train[(train['compliance'] == 1) | (train['compliance'] == 0)]
train

,violation_code,judgment_amount,compliance,lat,lon,Total_amount_to_pay,time_gap,disposition_Not responsible by Determination,disposition_Not responsible by Dismissal,disposition_PENDING JUDGMENT,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,disposition_SET-ASIDE (PENDING JUDGMENT),agency_name_Department of Public Works,agency_name_Detroit Police Department,agency_name_Health Department,agency_name_Neighborhood City Halls
ticket_id,,,,,,,,,,,,,,,,,,,
22056,202,305.0,0.0,42.390729,-83.124268,305.0,369.0,0,0,0,0,0,1,0,0,0,0,0,0
27586,116,855.0,1.0,42.326937,-83.135118,855.0,378.0,0,0,0,0,0,0,1,0,0,0,0,0
22046,202,305.0,0.0,42.145257,-83.208233,305.0,323.0,0,0,0,0,0,1,0,0,0,0,0,0
18738,115,855.0,0.0,42.433466,-83.023493,855.0,253.0,0,0,0,0,0,1,0,0,0,0,0,0
18735,114,140.0,0.0,42.388641,-83.037858,140.0,251.0,0,0,0,0,0,1,0,0,0,0,0,0
18733,114,140.0,0.0,42.388641,-83.037858,140.0,251.0,0,0,0,0,0,1,0,0,0,0,0,0
28204,116,855.0,0.0,42.435773,-82.963348,855.0,323.0,0,0,0,0,0,1,0,0,0,0,0,0
18743,116,855.0,0.0,42.395765,-83.022333,855.0,209.0,0,0,0,0,0,1,0,0,0,0,0,0
18741,116,855.0,0.0,42.440190,-83.180488,855.0,201.0,0,0,0,0,0,1,0,0,0,0,0,0


In [56]:
train.columns

Index(['violation_code', 'judgment_amount', 'compliance', 'lat', 'lon',
       'Total_amount_to_pay', 'time_gap',
       'disposition_Not responsible by Determination',
       'disposition_Not responsible by Dismissal',
       'disposition_PENDING JUDGMENT',
       'disposition_Responsible (Fine Waived) by Deter',
       'disposition_Responsible by Admission',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination',
       'disposition_SET-ASIDE (PENDING JUDGMENT)',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department',
       'agency_name_Health Department', 'agency_name_Neighborhood City Halls'],
      dtype='object')

In [58]:
y = train['compliance']
X = train.drop('compliance', axis = 1)

Before submitting it to autograder let's test which model is actully better I am reducing dataset to around 50k (to increse speed) from training data (as we have correct values here) 

In [76]:
train_sample = train.sample(frac=0.4, random_state=0)

y_sample = train_sample['compliance']
X_sample = train_sample.drop('compliance', axis = 1)

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample)

This is a the Base classifier any model shouldnot do worse than this

In [86]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score

dummy_clf = DummyClassifier(strategy = 'most_frequent').fit(X_train,y_train)
ypred = dummy_clf.predict(X_test)

print('Dummy Classifier:\n')
print('Test Accuracy = {:.3f}'.format(dummy_clf.score(X_test, y_test)))
print('Train Accuracy = {:.3f}'.format(dummy_clf.score(X_train, y_train)))
print('AUC = {:.3f}'.format(roc_auc_score(y_test, ypred)))

Dummy Classifier:

Test Accuracy = 0.926
Train Accuracy = 0.929
AUC = 0.500


In [88]:
from sklearn.naive_bayes import GaussianNB

nbclf = GaussianNB().fit(X_train, y_train)
ypred = nbclf.predict(X_test)

print('Naive Bayes Classifier:\n')
print('Test Accuracy = {:.3f}'.format(nbclf.score(X_test, y_test)))
print('Train Accuracy = {:.3f}'.format(nbclf.score(X_train, y_train)))
print('AUC = {:.3f}'.format(roc_auc_score(y_test, ypred)))

Naive Bayes Classifier:

Test Accuracy = 0.849
Train Accuracy = 0.854
AUC = 0.708


Now let's try other classifiers to choose the best one for actual dataset

In [94]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(random_state =0).fit(X_train_scaled,y_train)
ypred = svm.predict(X_test_scaled)

print('SVC Classifier:\n')
print('Test Accuracy = {:.3f}'.format(svm.score(X_test_scaled, y_test)))
print('Train Accuracy = {:.3f}'.format(svm.score(X_train_scaled, y_train)))
print('AUC = {:.3f}'.format(roc_auc_score(y_test, ypred)))

SVC Classifier:

Test Accuracy = 0.931
Train Accuracy = 0.932
AUC = 0.531


SVM takes a lot of time

In [95]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

reg = MLPRegressor(hidden_layer_sizes=(20,20),random_state = 0).fit(X=X_train_scaled,y = y_train)
ypred = reg.predict(X_test_scaled)

print('Neural Network Regressor:\n')
print('Test Accuracy = {:.3f}'.format(reg.score(X_test_scaled, y_test)))
print('Train Accuracy = {:.3f}'.format(reg.score(X_train_scaled, y_train)))
print('AUC = {:.3f}'.format(roc_auc_score(y_test, ypred)))

Neural Network Regressor:

Test Accuracy = 0.122
Train Accuracy = 0.136
AUC = 0.758


In [96]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(X_train,y_train)
ypred = linreg.predict(X_test)

print('Linear Regressor:\n')
print('Test Accuracy = {:.3f}'.format(linreg.score(X_test, y_test)))
print('Train Accuracy = {:.3f}'.format(linreg.score(X_train, y_train)))
print('AUC = {:.3f}'.format(roc_auc_score(y_test, ypred)))

Linear Regressor:

Test Accuracy = 0.126
Train Accuracy = 0.145
AUC = 0.768


In [98]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(max_depth = 6, random_state=0).fit(X_train, y_train)
ypred = reg.predict(X_test)

print('Random Forrest Regressor:\n')
print('Test Accuracy = {:.3f}'.format(reg.score(X_test, y_test)))
print('Train Accuracy = {:.3f}'.format(reg.score(X_train, y_train)))
print('AUC = {:.3f}'.format(roc_auc_score(y_test, ypred)))

Random Forrest Regressor:

Test Accuracy = 0.181
Train Accuracy = 0.209
AUC = 0.789


So From above analysis ,we can use randomforest or GaussianNB for best results,any classifier except SVMs should work maybe svm requires more tuning or maybe more data. I will be using RandomForest for my actual submission

Actual code actually has slight modification due to grader issues,But concepts used are same Below code passes the grader with 100 points